In [15]:
import os, sys

import geopandas as gpd

from shapely.geometry import LineString

In [16]:
LineString?

In [54]:
inputEA = r"C:\Users\WB411133\OneDrive - WBG\AAA_BPS\GOST\Projects\Ghana_Census_Support\Data\GSS_Data\UNEDITED_DSITRICTS\2020 BR ASUNAFO SOUTH1\asunafo south final.shp"
inE = gpd.read_file(inputEA)
inE['geometry'] = inE.buffer(0)
inE.head()

,SP_ID,GEOMETRY_S,ID1,EST_POP,Z00_EA_COD,REG_NAME,DIST_CODE,DIST_NAME,TYP,LOC_NAME,...,FID_1,Z020_LOC,Z020_TYPE,Z020_ESTP,Z020_SA,Z020_EA,SPECIAL,Z020_PCOD,2020_BASE,geometry
0,11306,None,0,None,0701001785,BRONG AHAFO,0701,ASUNAFO SOUTH,3,KESEKROM (KWADWO OWAN0,...,11306,KESEKROM (KWADWO,3,271,None,152,None,0701100152,KESEKROM (KWADWO DWAN),"POLYGON ((373249.594 739330.833, 373379.690 73..."
1,11307,None,0,None,0701001784,BRONG AHAFO,0701,ASUNAFO SOUTH,3,TANOSO,...,11307,TANOSO,3,383,None,148,None,0701100148,TANOSO,"POLYGON ((388759.313 739967.583, 388775.813 73..."
2,11308,None,0,None,0701001701,BRONG AHAFO,0701,ASUNAFO SOUTH,1,ASAWENSO,...,11308,ASAWENSO,1,658,None,001,None,0701100001,ASAWENSO,"POLYGON ((376889.852 615176.881, 376973.505 61..."
3,11310,None,0,None,0701001703,BRONG AHAFO,0701,ASUNAFO SOUTH,1,AGYEIKROM,...,11310,AGYEIKROM,1,680,None,005,None,0701100005,AGYEIKROM,"POLYGON ((372468.662 631752.987, 373125.803 63..."
4,11311,None,0,None,0701001704,BRONG AHAFO,0701,ASUNAFO SOUTH,3,KOKOOSO No. 3,...,11311,KOKOOSO No. 3,3,677,None,006,None,0701100006,KOKOOSO N0.3,"POLYGON ((378066.813 617074.397, 377954.653 61..."


In [46]:
from math import atan2, cos, sin, degrees



def findNext(curShape, otherEAs, direction):
    ''' find the next EA in the order of EAs using a serpentine search

    INPUT
    curShape [ shapely polygon ] - current EA
    otherEAs [ geopandas dataframe ] - EAS from which to choose next EA
    direction [ number ] - direction to search in: 1 is right, 2 is up, 3 is left
    curNum [ number ] - current EA number

    RETURNS
    ???
    '''
    #Find EAs that intersect current EA
    possibleEAs = otherEAs[otherEAs.intersect(curShape)]
    
    #Identify the EAs in the correct direction
    possibleEAs['DIRECTION'] = possibleEAs['geometry'].apply(lambda x: getDirection(x, curShape))
    #Select the optimal SA
    if direction == 1: # right
        minDirection = 90
        maxDirection = 180
    elif direction 
    

def getSerpentineOrder(inD, loc_col):
    ''' The Ghana census numbers localities and EAs in a serpentine manner, starting in the lower left.
    EAs within localities are numbered in a serpentine manner within that locality. The process is to number 
    localities in serpentine order. If the locality has more than 1 EA in it, number those EAs in a serpentine order
    
    INPUT
    inD [ geopandas dataframe ] - the input EA file
    
    RETURNS [ geopandas dataframe ] - same as the input, but with two new fields, EA_NUM, and LOC_NUM
    '''       
    ###Find the lower left EA to start with
    # Create a line along the bottom of the bounds
    inD = inE
    b = inD.total_bounds
    bottomLine = LineString([b[0], b[1]], [b[2], b[1]])
    
    #Measure distance from linestring to boundaries
    inD['bottom_distance'] = inD['geometry'].apply(lambda x: x.distance(bottomLine))
    
    startingEA = inD[inD['bottom_distance'].idxmin()]
    
    
        

In [35]:
loc_column = "LOC_NAME"

inLoc = inE.dissolve(by=loc_column)


In [48]:
shp1 = inLoc['geometry'].iloc[1]
shp2 = inLoc['geometry'].iloc[2]
getDirection(shp1, shp2)

122.2141580304293

In [62]:
def getDirection(shp1, shp2):
    ''' get the direction from shp1 to shp2
    
    INPUT
    shp1, shp2 [ shapely polygon ]
    
    RETURNS
    [number] - direction from shp1 to shp2
    '''
    lon1 = shp1.centroid.x
    lat1 = shp1.centroid.y
    lon2 = shp2.centroid.x
    lat2 = shp2.centroid.y
    angle = atan2(cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos (lon2 - lon1), sin(lon2-lon1) * cos(lat2))
    bearing = (degrees(angle) + 360) % 360
    return(bearing)

focalEA = inE.iloc[0]
possibleEAs = inE[inE.intersects(focalEA['geometry'])]
#possibleEAs = possibleEAs[possibleEAs['SP_ID'] != focalEA['SP_ID']]
possibleEAs['DIRECTION'] = possibleEAs['geometry'].apply(lambda x: getDirection(focalEA['geometry'], x))
possibleEAs['geometry'] = possibleEAs['geometry'].apply(lambda x: x.centroid) 
possibleEAs.to_file("C:/temp/possibleEAs.shp")
focalEA

C:\WBG\Anaconda\envs\geog\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\WBG\Anaconda\envs\geog\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SP_ID                                                     11306
GEOMETRY_S                                                 None
ID1                                                           0
EST_POP                                                    None
Z00_EA_COD                                           0701001785
REG_NAME                                            BRONG AHAFO
DIST_CODE                                                  0701
DIST_NAME                                         ASUNAFO SOUTH
TYP                                                           3
LOC_NAME                                 KESEKROM (KWADWO OWAN0
Z010_EA_CO                                           0701100152
BASE_NAM                                 KESEKROM (KWADWO OWAN0
FID_1                                                     11306
Z020_LOC                                       KESEKROM (KWADWO
Z020_TYPE                                                     3
Z020_ESTP                               

In [57]:
possibleEAs.head()

,SP_ID,GEOMETRY_S,ID1,EST_POP,Z00_EA_COD,REG_NAME,DIST_CODE,DIST_NAME,TYP,LOC_NAME,...,Z020_LOC,Z020_TYPE,Z020_ESTP,Z020_SA,Z020_EA,SPECIAL,Z020_PCOD,2020_BASE,geometry,DIRECTION
0,11306,None,0,None,0701001785,BRONG AHAFO,0701,ASUNAFO SOUTH,3,KESEKROM (KWADWO OWAN0,...,KESEKROM (KWADWO,3,271,None,152,None,0701100152,KESEKROM (KWADWO DWAN),"POLYGON ((373249.594 739330.833, 373379.690 73...",0.000000
1,11307,None,0,None,0701001784,BRONG AHAFO,0701,ASUNAFO SOUTH,3,TANOSO,...,TANOSO,3,383,None,148,None,0701100148,TANOSO,"POLYGON ((388759.313 739967.583, 388775.813 73...",227.304031
86,11395,None,0,None,0701001783,BRONG AHAFO,0701,ASUNAFO SOUTH,1,YANKYE,...,YANKYE,1,368,None,151,None,0701100151,YANKYE,"POLYGON ((364320.204 727799.292, 364972.893 72...",230.214893
87,11396,None,0,None,0701001783,BRONG AHAFO,0701,ASUNAFO SOUTH,3,OP. OKURU AKURA,...,OP. OKURU AKURA,3,467,None,150,None,0701100150,OP. OKURA AKURA,"POLYGON ((366799.704 720524.521, 366665.594 72...",15.931847
88,11398,None,0,None,0701001795,BRONG AHAFO,0701,ASUNAFO SOUTH,3,DOMEABRA ( KWABENA KUMA ),...,DOMEABRA ( KWABE,3,501,None,178,None,0701100178,DOMEABRA (KWABENA KUMA),"POLYGON ((369397.047 742356.333, 370064.907 74...",356.893834


In [55]:
import pandas as pd

excel_file = "C:/temp/EA_Codes.xlsx"
inEA = pd.read_excel(excel_file)
inEA

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.